In [1]:
#!pip install blosc dopamine-rl transformers==4.11 --user

In [1]:
import sys
import os
os.environ["LD_LIBRARY_PATH"]="/.mujoco/mujoco210/bin"

import gym
import numpy as np
import torch
import wandb
import matplotlib.pyplot as plt
import mem_transformer
from tqdm import tqdm

import argparse
import pickle
import random
import sys
import os
import glob
import d4rl
from colabgymrender.recorder import Recorder
sys.path.append('/root/RMDT_paper/rmdt_transformer/gym/')
from decision_transformer.evaluation.evaluate_episodes import evaluate_episode, evaluate_episode_rtg

from utils.eval_functions import get_batch, get_returns

/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
/usr/local/lib/python3.10/dist-packages/mujoco_py/builder.py:9: DeprecationWarning: The distutils.sysconfig module is deprecated, use sysconfig instead
  from distutils.sysconfig import customize_compiler
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
libEGL warning: egl: failed to create dri2 screen
libEGL warning: egl: failed to create dri2 screen
pybullet build time: Jul 28 2023 09:08:37
/usr/local/lib/python3.10/dist-packages/colabgymrender/recorder.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display

In [2]:
envs = [['halfcheetah-medium-v2','checkpoints_Cheetah_Med',17,6,1000,6000],
        ['halfcheetah-medium-replay-v2','checkpoints_Cheetah_Med_Repl',17,6,1000,6000],
        ['halfcheetah-expert-v2','checkpoints_Cheetah_Expert',17,6,1000,6000],
       
        ['walker2d-medium-v2','checkpoints_Walker_Med',17,6,1000,5000],
        ['walker2d-medium-replay-v2','checkpoints_Walker_Med_Repl',17,6,1000,5000],
        ['walker2d-expert-v2','checkpoints_Walker_Expert',17,6,1000,5000],
        
        ['hopper-medium-v2','checkpoints_Hopper_Med',11,3,1000,3600],
        ['hopper-medium-replay-v2','checkpoints_Hopper_Med_Repl',11,3,1000,3600],
        ['hopper-expert-v2','checkpoints_Hopper_Expert',11,3,1000,3600],
       ]

In [3]:
gym_name = envs[0]
device = torch.device('cuda:0')

In [4]:
game_gym_name = gym_name[0]
env_name = game_gym_name.split('-')[0]
dataset = '-'.join(game_gym_name.split('-')[1:-1])
state_dim = gym_name[2]
act_dim = gym_name[3]
max_ep_len = gym_name[4]
ret_global = gym_name[5]
use_recorder = False

env = gym.make(game_gym_name)
directory = '../video'
if use_recorder:
    env = Recorder(env, directory, fps=30)

# load dataset
dataset_path = f'../data/{env_name}-{dataset}-v2.pkl'
with open(dataset_path, 'rb') as f:
    trajectories = pickle.load(f)
states, traj_lens, returns = [], [], []
for path in trajectories:
    states.append(path['observations'])
    traj_lens.append(len(path['observations']))
    returns.append(path['rewards'].sum())
traj_lens, returns = np.array(traj_lens), np.array(returns)  
num_timesteps = sum(traj_lens)
states = np.concatenate(states, axis=0)
state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
state_mean_torch, state_std_torch = torch.from_numpy(state_mean).to(device), torch.from_numpy(state_std).to(device) 

print('=' * 50)
print(f'Starting new experiment: {env_name} {dataset}')
print(f'{len(traj_lens)} trajectories, {num_timesteps} timesteps found')
print(f'Average return: {np.mean(returns):.2f}, std: {np.std(returns):.2f}')
print(f'Max return: {np.max(returns):.2f}, min: {np.min(returns):.2f}')
print('=' * 50)

  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")



Starting new experiment: halfcheetah medium
1000 trajectories, 1000000 timesteps found
Average return: 4770.33, std: 355.75
Max return: 5309.38, min: -310.23


In [5]:
state_mean = state_mean*0.+1
state_std = state_std*0.+1
state_mean_torch = state_mean_torch*0.+1
state_std_torch = state_std_torch*0.+1

In [6]:
K = 20*3
batch_size = 4096
num_eval_episodes = 10
pct_traj = 1.
max_ep_len = 10000
scale = 1000.

In [7]:
# only train on top pct_traj trajectories (for %BC experiment)
num_timesteps = max(int(pct_traj*num_timesteps), 1)
sorted_inds = np.argsort(returns)  # lowest to highest
num_trajectories = 1
timesteps = traj_lens[sorted_inds[-1]]
ind = len(trajectories) - 2
while ind >= 0 and timesteps + traj_lens[sorted_inds[ind]] <= num_timesteps:
    timesteps += traj_lens[sorted_inds[ind]]
    num_trajectories += 1
    ind -= 1
sorted_inds = sorted_inds[-num_trajectories:]
# used to reweight sampling so we sample according to timesteps instead of trajectories
p_sample = traj_lens[sorted_inds] / sum(traj_lens[sorted_inds])

s, a, r, d, rtg, timesteps, mask = get_batch(trajectories,state_mean,state_std,num_trajectories,max_ep_len,state_dim,act_dim,sorted_inds,p_sample,device)



In [8]:
epochs = 20
vocab_size = 100
n_layer = 3
n_head = 1
d_model = 128
d_head = 128
d_inner = 128
dropout = 0.1
dropatt = 0.05
MEM_LEN = 2 ########################### 2 FOR DTXL 0
ext_len = 1
tie_weight = False
num_mem_tokens = 3*5 ########################### 3*5 FOR DTXL 0
mem_at_end = True ########################### True FOR DTXL False
learning_rate = 1e-4
weight_decay = 1e-4
warmup_steps = 100
grad_norm_clip = 1.0
max_timestep = 10000
context_length = 20
sections = 3
block_size = 3*context_length
num_spets_per_epoch = 1000
EFFECTIVE_SIZE_BLOCKS = context_length * sections
BLOCKS_CONTEXT = block_size//3
is_train = True
wwandb = True

In [9]:
model = mem_transformer.MemTransformerLM(STATE_DIM=state_dim,ACTION_DIM=act_dim,n_token=vocab_size, n_layer=n_layer, n_head=n_head, d_model=d_model,
        d_head=d_head, d_inner=d_inner, dropout=dropout, dropatt=dropatt, mem_len=MEM_LEN, ext_len=ext_len, 
                                         tie_weight=tie_weight, num_mem_tokens=num_mem_tokens, mem_at_end=mem_at_end)
torch.nn.init.xavier_uniform_(model.r_w_bias);
torch.nn.init.xavier_uniform_(model.r_r_bias);
model.train()
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate,  weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda steps: min((steps+1)/warmup_steps, 1))
raw_model = model.module if hasattr(model, "module") else model

In [10]:
if wwandb:
    idd = 'tzuzlv1i' #
    wandb.init(project="rnd_habitat", name='RMDT_chetah_MED_T1_Original', save_code=True, resume="allow")
    wwandb = True

  from jsonschema import FormatChecker, RefResolver, validators

  from jsonschema import Draft7Validator, RefResolver, ValidationError

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alstar (cds). Use `wandb login --relogin` to force relogin
  from IPython.core.display import display, HTML  # type: ignore



In [ ]:
ckpt_path = '../checkpoints/checkpoints_Hopper_MedExp_orgn/'

isExist = os.path.exists(ckpt_path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(ckpt_path)
    
files = glob.glob(ckpt_path+'*')
for f in files:
    os.remove(f)

INTERVAL = 100
losses = []
wandb_step  = 0
for epoch in range(epochs):
    pbar = tqdm(enumerate(list(range(num_spets_per_epoch))), total=num_spets_per_epoch)
    for it, i in pbar:

        s, a, r, d, rtg, timesteps, mask = get_batch(trajectories,state_mean,state_std,num_trajectories,max_ep_len,state_dim,act_dim,sorted_inds,p_sample,device,batch_size=batch_size)

        s = s[torch.all(mask==1,dim=1)]
        a = a[torch.all(mask==1,dim=1)]
        rtg = rtg[torch.all(mask==1,dim=1)]
        timesteps = timesteps[torch.all(mask==1,dim=1)]

        memory = None
        mem_tokens=None
        for block_part in range(EFFECTIVE_SIZE_BLOCKS//BLOCKS_CONTEXT):

            from_idx = block_part*(BLOCKS_CONTEXT)
            to_idx = (block_part+1)*(BLOCKS_CONTEXT)
            x1 = s[:, from_idx:to_idx, :].to(device)
            y1 = a[:, from_idx:to_idx, :].to(device)
            r1 = rtg[:,:-1,:][:, from_idx:to_idx, :].to(device) ############ SWITCH TO RTG
            t1 = timesteps[:, from_idx:to_idx].to(device)

            if mem_tokens is not None:
                mem_tokens = mem_tokens.detach()
            elif raw_model.mem_tokens is not None:
                mem_tokens = raw_model.mem_tokens.repeat(1, x1.shape[0], 1)

            with torch.set_grad_enabled(is_train):

                if memory is not None:
                    res = model(x1, y1, r1, y1,t1, *memory, mem_tokens=mem_tokens)
                else:
                    res = model(x1, y1, r1, y1,t1, mem_tokens=mem_tokens)
                memory = res[0][2:]
                logits, loss = res[0][0], res[0][1]
                losses.append(loss.item())
                print('Loss: ',loss.item())
                if wwandb:
                    wandb.log({"train_loss":  loss.item()})

            if is_train:
                # backprop and update the parameters
                model.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_norm_clip)
                optimizer.step()
                scheduler.step()
                #print()

                # decay the learning rate based on our progress
                lr = optimizer.state_dict()['param_groups'][0]['lr']

                # report progress
                pbar.set_description(f"epoch {epoch+1} iter {it}: train loss {loss.item():.5f}. lr {lr:e}")
                  

        if it % INTERVAL == 0:
            #"""
            rews = []
            steps = 10
            prompt_steps = 1
            pbar = tqdm(enumerate(list(range(steps))), total=steps)
            for it, i in pbar:
                eval_return, env = get_returns(model, env,ret_global, context_length, state_dim, act_dim, state_mean_torch, state_std_torch, max_ep_len, device, prompt_steps=prompt_steps, memory_20step=True, without_memory=False)
                rews.append(eval_return)
                    
            print(f"Prompt_steps: {prompt_steps}  Mean Reward: {np.mean(np.array(rews)):.5f}. STD Reward: {np.std(np.array(rews)):.5f}")        
            if wwandb:            
                wandb.log({"episode_STD":  np.std(rews)})#, step=wandb_step)
                wandb.log({"episode_MEAN":  np.mean(rews)})#, step=wandb_step)

            rews = []
            steps = 10
            prompt_steps = 20
            pbar = tqdm(enumerate(list(range(steps))), total=steps)
            for it, i in pbar:
                eval_return, env = get_returns(model, env,ret_global, context_length, state_dim, act_dim, state_mean_torch, state_std_torch, max_ep_len, device, prompt_steps=prompt_steps, memory_20step=True, without_memory=False)
                rews.append(eval_return)
                    
            print(f"Prompt_steps: {prompt_steps}  Mean Reward: {np.mean(np.array(rews)):.5f}. STD Reward: {np.std(np.array(rews)):.5f}")        
            if wwandb:            
                wandb.log({"20_episode_STD":  np.std(rews)}) #, step=wandb_step)
                wandb.log({"20_episode_MEAN":  np.mean(rews)}) #, step=wandb_step)
            #"""    
            
            #if wwandb:
            #    wandb.log({"train_loss":  loss.item()}, step=wandb_step)
                
            wandb_step += 1     
            torch.save(model.state_dict(), ckpt_path+'_'+str(wandb_step)+'_HOPPER_medexp.pth')
    

epoch 1 iter 0: train loss 0.78773. lr 2.000000e-06:   0%|          | 0/1000 [00:05<?, ?it/s]

Loss:  0.7877335548400879


epoch 1 iter 0: train loss 0.78442. lr 3.000000e-06:   0%|          | 0/1000 [00:05<?, ?it/s]

Loss:  0.784415602684021
Loss:  0.7838305830955505


epoch 1 iter 0: train loss 0.78383. lr 4.000000e-06:   0%|          | 0/1000 [00:05<?, ?it/s]
 10%|█         | 1/10 [00:05<00:52,  5.86s/it]

...

Loss:  0.032302942126989365
Loss:  0.03238118812441826


epoch 7 iter 0: train loss 0.03268. lr 1.000000e-04:  91%|█████████ | 907/1000 [1:32:04<13:32,  8.73s/it]

Loss:  0.03245532512664795
Loss:  0.032375115901231766
Loss:  0.03199220076203346


epoch 7 iter 0: train loss 0.03268. lr 1.000000e-04:  91%|█████████ | 908/1000 [1:32:09<11:26,  7.47s/it]

Loss:  0.032205935567617416
Loss:  0.032467275857925415
Loss:  0.030944494530558586


In [ ]:
i

# Eval

In [13]:
steps = 10
print('#######################################: '+'memory_20step=True, without_memory=False')
for prompt_steps in [1,5,20]:
    rews = []
    pbar = tqdm(enumerate(list(range(steps))), total=steps)
    for it, i in pbar:
        eval_return, env = get_returns(model, env,ret_global, context_length, state_dim, act_dim, state_mean_torch, state_std_torch, max_ep_len, device, prompt_steps=prompt_steps, memory_20step=True, without_memory=False)
        rews.append(eval_return)
    print(f"Prompt_steps: {prompt_steps}  Mean Reward: {np.mean(np.array(rews)):.5f}. STD Reward: {np.std(np.array(rews)):.5f}")   


#######################################: memory_20step=True, without_memory=False


 10%|█         | 1/10 [00:00<00:05,  1.72it/s]

target return: 3, eval return: 12


 20%|██        | 2/10 [00:01<00:04,  1.74it/s]

target return: 3, eval return: 12


...

target return: 3, eval return: 62


 90%|█████████ | 9/10 [00:23<00:02,  2.99s/it]

target return: 3, eval return: 73


100%|██████████| 10/10 [00:23<00:00,  2.39s/it]

target return: 3, eval return: 21
Prompt_steps: 20  Mean Reward: 48.39621. STD Reward: 19.53681


In [ ]:
eval_return

In [15]:
torch.save(model.state_dict(), 'cheetah_medium_RMT_100_1000.pth')

In [ ]:
SCALE=1000.
ret = args.ret_global/SCALE
T_rewards, T_norm_rewards = [], []
done = True
frames = []
HISTORY_LEN = args.context_length#(MEM_SEGMENTS+1)*CONTEXT_LEN # EFFECTIVE_SIZE_BLOCKS
N = 1
for i in range(N):
    state = env.reset()
    states = (torch.from_numpy(state).reshape(1, args.state_dim).to(device=device, dtype=torch.float32).unsqueeze(0)- args.state_mean_torch) / args.state_std_torch
    actions = torch.zeros(1,args.act_dim).to(device).unsqueeze(0)
    rtgs = torch.tensor([ret], dtype=torch.float).to(device).unsqueeze(0).unsqueeze(0)
    timesteps = torch.zeros((1, 1), dtype=torch.int64).to(device)
    if args.num_mem_tokens > 0:
        mem_tokens = model.mem_tokens.repeat(1, 1, 1).detach()
    else:
        mem_tokens = None
    saved_context = None
    segment = 0
    episode_return = 0
    episode_length = 0

    for t in range(args.max_ep_len):

        if t%20==0 and t>5:
            mem_tokens = new_mem
            saved_context = new_notes
            states = states[:,:1,:]
            actions = actions[:,:1,:]
            rtgs = rtgs[:,:1,:]
            timesteps = timesteps[:,:1]
    
        sampled_action, new_mem, new_notes = sample(
                    model=model,
                    x=states,
                    block_size=HISTORY_LEN,
                    steps=1,
                    temperature=1.0,
                    sample=True,
                    actions=actions, 
                    rtgs=rtgs, 
                    timesteps=timesteps,
                    mem_tokens=mem_tokens,
                    saved_context=saved_context)
        actions = torch.cat([actions, sampled_action.unsqueeze(0)],dim=1)
        action = sampled_action.cpu().numpy()[0][0]
        state, reward, done, _ = env.step(action)
        #print(reward)
        cur_state = (torch.from_numpy(state).to(device=device).reshape(1, args.state_dim).float().unsqueeze(0)- args.state_mean_torch) / args.state_std_torch
        states = torch.cat([states, cur_state], dim=1)
        pred_return = rtgs[0,-1] - reward/SCALE
        #print(env.get_normalized_score(reward))
        rtgs = torch.cat([rtgs, pred_return.reshape(1, 1, 1)], dim=1)
        timesteps = torch.cat([timesteps,torch.ones((1, 1), device=device, dtype=torch.long) * (t+1)], dim=1)

        episode_return += reward
        episode_length += 1

    NORM_episode_return = env.get_normalized_score(episode_return)*100.
    T_rewards.append(episode_return) 
    T_norm_rewards.append(NORM_episode_return)

print(np.array(T_rewards).mean(),np.array(T_rewards).std())
print(np.array(T_norm_rewards).mean(),np.array(T_norm_rewards).std())
    

In [ ]:
memory = None
mem_tokens=None
for block_part in range(config.EFFECTIVE_SIZE_BLOCKS//config.BLOCKS_CONTEXT):
    from_idx = block_part*(config.BLOCKS_CONTEXT)
    to_idx = (block_part+1)*(config.BLOCKS_CONTEXT)
    x1 = s[:, from_idx:to_idx, :].to(self.device)
    y1 = a[:, from_idx:to_idx, :].to(self.device)
    r1 = rtg[:, from_idx:to_idx, :].to(self.device)
    t1 = timesteps[:, from_idx:to_idx].to(self.device)

    if mem_tokens is not None:
        mem_tokens = mem_tokens.detach()
    elif raw_model.mem_tokens is not None:
        mem_tokens = raw_model.mem_tokens.repeat(1, x1.shape[0], 1)
            
    with torch.set_grad_enabled(is_train):

        print('Shape: ',x1.shape,y1.shape,r1.shape,t1.shape,mem_tokens.shape)
        
        if memory is not None:
            res = model(x1, y1, r1, y1, t1, *memory, mem_tokens=mem_tokens) # timesteps = None
        else:
            res = model(x1, y1, r1, y1, t1, mem_tokens=mem_tokens)